# Comparing the 2024 and 2025 Michelin Guide to France

In [41]:
import pandas as pd

In [42]:
france_24 = pd.read_csv('../../../2024/data/France/all_restaurants(arrondissements).csv')
france_25 = pd.read_csv('../../data/France/all_restaurants(arrondissements).csv')

In [43]:
france_24.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1017 entries, 0 to 1016
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   name            1017 non-null   object 
 1   address         1017 non-null   object 
 2   location        1017 non-null   object 
 3   arrondissement  1017 non-null   object 
 4   department_num  1017 non-null   object 
 5   department      1017 non-null   object 
 6   capital         1017 non-null   object 
 7   region          1017 non-null   object 
 8   price           1017 non-null   object 
 9   cuisine         1017 non-null   object 
 10  url             973 non-null    object 
 11  award           1017 non-null   object 
 12  stars           1017 non-null   float64
 13  longitude       1017 non-null   float64
 14  latitude        1017 non-null   float64
dtypes: float64(3), object(12)
memory usage: 119.3+ KB


In [44]:
france_25.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2985 entries, 0 to 2984
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   name            2985 non-null   object 
 1   address         2985 non-null   object 
 2   location        2985 non-null   object 
 3   arrondissement  2978 non-null   object 
 4   department_num  2985 non-null   object 
 5   department      2985 non-null   object 
 6   capital         2985 non-null   object 
 7   region          2985 non-null   object 
 8   price           2985 non-null   object 
 9   cuisine         2985 non-null   object 
 10  url             2831 non-null   object 
 11  award           2985 non-null   object 
 12  stars           2985 non-null   float64
 13  greenstar       2985 non-null   int64  
 14  longitude       2985 non-null   float64
 15  latitude        2985 non-null   float64
dtypes: float64(3), int64(1), object(12)
memory usage: 373.3+ KB


In [45]:
non_selected_25 = france_25[france_25['stars'] != 0.25]
print(len(non_selected_25))

1042


----
&nbsp;
## Identify new or removed entries focusing on 1+ star restaurants

In [46]:
# Filter france_24 and france_25 for star ratings >= 1
subset_24 = france_24[france_24['stars'] >= 1][['name', 'address', 'location', 'url', 'stars']]
subset_25 = france_25[france_25['stars'] >= 1][['name', 'address', 'location', 'url', 'stars']]

# Turn the filtered data into sets of tuples
set_subset_24 = set(tuple(x) for x in subset_24.values)
set_subset_25 = set(tuple(x) for x in subset_25.values)

# Find new and closed Michelin stars
potential_new_stars = set_subset_25 - set_subset_24
potential_closed_stars = set_subset_24 - set_subset_25

In [47]:
print(f'Potentially entered the guide:\n{len(potential_new_stars)} starred restaurants in 2025 guide not in 2024')
print(f'\nPotentially left the guide:\n{len(potential_closed_stars)} starred restaurants in 2024 guide not in 2025')

Potentially entered the guide:
191 starred restaurants in 2025 guide not in 2024

Potentially left the guide:
177 starred restaurants in 2024 guide not in 2025


----
&nbsp;
### Using Pandas method
#### The above does not tell us how the distribution has changed.

In [48]:
# Now find name changes. For this, we need to find common addresses with different names
common_locations = set(subset_24['url']).intersection(set(subset_25['url']))

# Filtering out the common url from both DataFrames
common_24 = subset_24[subset_24['url'].isin(common_locations)]
common_25 = subset_25[subset_25['url'].isin(common_locations)]

In [49]:
# Merge the data frames on 'url' and 'location'
merged_df = pd.merge(common_24, common_25, on=['url', 'location'], suffixes=('_24', '_25'))

# Filter entries where names or star ratings have changed
name_star_changes = merged_df[(merged_df['name_24'] != merged_df['name_25']) | (merged_df['stars_24'] != merged_df['stars_25'])]

# Rename columns to reflect the data properly
name_star_changes = name_star_changes[['name_24', 'name_25', 'address_24', 'address_25', 'location', 'stars_24', 'stars_25']]

In [50]:
name_star_changes

,name_24,name_25,address_24,address_25,location,stars_24,stars_25
24,Georges Blanc,Georges Blanc,Place du Marché,Place du Marché,"Vonnas, 01540",3.0,2.0
30,La Table des Amis by Christophe Bacquié,La Table des Amis,"Les Mas Les Eydins, 2420 chemin du Four","Les Mas Les Eydins, 2420 chemin du Four","Bonnieux, 84480",2.0,2.0
33,L'Abysse au Pavillon Ledoyen,Pavyllon,8 avenue Dutuit,8 avenue Dutuit,"Paris, 75008",2.0,1.0
51,Le Puits Saint Jacques,Le Puits Saint Jacques,57 avenue Victor-Capoul,57 avenue Victor-Capoul,"Pujaudran, 32600",2.0,1.0
75,Christopher Coutanceau,Christopher Coutanceau,Plage de la Concurrence,Plage de la Concurrence,"La Rochelle, 17000",2.0,3.0
94,Le Grand Contrôle,Ducasse au Château de Versailles - Le Grand Co...,12 rue de l'Indépendance-Américaine,12 rue de l'Indépendance-Américaine,"Versailles, 78000",1.0,1.0
113,Maison Nouvelle,Maison Nouvelle,11 rue Rode,11 rue Rode,"Bordeaux, 33000",1.0,2.0
119,Chakaiseiki Akiyoshi,Chakaiseki Akiyoshi,59 rue Letellier,59 rue Letellier,"Paris, 75015",1.0,1.0
158,Ekaitza,Ekaitza,15 quai Maurice-Ravel,15 quai Maurice-Ravel,"Ciboure, 64500",1.0,2.0
175,Rozó,Rozó,34 rue Raymond-Derain,34 rue Raymond-Derain,"Marcq-en-Barœul, 59700",1.0,2.0


We're focusing on the changes from ⭐️$\implies$⭐️⭐️ and from ⭐️⭐️$\implies$⭐️⭐️⭐️ (or significant demotions)

We assign a similarity score to the different names and change `france_23` in place

In [51]:
from fuzzywuzzy import fuzz

# Function to preprocess and compare names
def preprocess(text):
    # Basic preprocessing: lowercasing and stripping spaces
    return text.lower().strip()

def compare_names(name1, name2):
    name1 = preprocess(name1)
    name2 = preprocess(name2)
    return fuzz.token_sort_ratio(name1, name2)  # Comparing with sorted tokens to handle reordering

# Applying the comparison across the DataFrame rows
name_star_changes['similarity_score'] = name_star_changes.apply(lambda x: compare_names(x['name_24'], x['name_25']), axis=1)

In [52]:
name_star_changes

,name_24,name_25,address_24,address_25,location,stars_24,stars_25,similarity_score
24,Georges Blanc,Georges Blanc,Place du Marché,Place du Marché,"Vonnas, 01540",3.0,2.0,100
30,La Table des Amis by Christophe Bacquié,La Table des Amis,"Les Mas Les Eydins, 2420 chemin du Four","Les Mas Les Eydins, 2420 chemin du Four","Bonnieux, 84480",2.0,2.0,62
33,L'Abysse au Pavillon Ledoyen,Pavyllon,8 avenue Dutuit,8 avenue Dutuit,"Paris, 75008",2.0,1.0,39
51,Le Puits Saint Jacques,Le Puits Saint Jacques,57 avenue Victor-Capoul,57 avenue Victor-Capoul,"Pujaudran, 32600",2.0,1.0,100
75,Christopher Coutanceau,Christopher Coutanceau,Plage de la Concurrence,Plage de la Concurrence,"La Rochelle, 17000",2.0,3.0,100
94,Le Grand Contrôle,Ducasse au Château de Versailles - Le Grand Co...,12 rue de l'Indépendance-Américaine,12 rue de l'Indépendance-Américaine,"Versailles, 78000",1.0,1.0,50
113,Maison Nouvelle,Maison Nouvelle,11 rue Rode,11 rue Rode,"Bordeaux, 33000",1.0,2.0,100
119,Chakaiseiki Akiyoshi,Chakaiseki Akiyoshi,59 rue Letellier,59 rue Letellier,"Paris, 75015",1.0,1.0,97
158,Ekaitza,Ekaitza,15 quai Maurice-Ravel,15 quai Maurice-Ravel,"Ciboure, 64500",1.0,2.0,100
175,Rozó,Rozó,34 rue Raymond-Derain,34 rue Raymond-Derain,"Marcq-en-Barœul, 59700",1.0,2.0,100


In [53]:
# Filter rows where the similarity score is greater than 65 (by inspection)
high_similarity_rows = name_star_changes[name_star_changes['similarity_score'] >= 65]

In [54]:
# Iterate over these rows to update the names in france_24
for _, row in high_similarity_rows.iterrows():
    # Update france_24 where the url and location match
    france_24.loc[(france_24['address'] == row['address_24']) & (france_25['location'] == row['location']), 'name'] = row['name_25']

---
&nbsp;
### New ⭐⭐⭐ Restaurants, New ⭐⭐ Restaurants and demotions

In [55]:
# Identify New Two-Star and Three-Star Restaurants
# Filter france_24 and france_25 for star ratings >= 1
subset_24 = france_24[france_24['stars'] >= 1][['name', 'address', 'location', 'url', 'stars']]
subset_25 = france_25[france_25['stars'] >= 1][['name', 'address', 'location', 'url', 'stars']]

In [56]:
# Now find name changes. For this, we need to find common addresses with different names
common_locations = set(subset_24['name']).intersection(set(subset_25['name']))

# Filtering out the common url from both DataFrames
common_24 = subset_24[subset_24['name'].isin(common_locations)]
common_25 = subset_25[subset_25['name'].isin(common_locations)]

In [57]:
# Merge the data frames on 'name' and 'location'
merged_df = pd.merge(common_24, common_25, on=['name', 'location', 'url'], suffixes=('_24', '_25'))

In [58]:
# Filter entries where names or star ratings have changed
star_changes = merged_df[(merged_df['stars_24'] != merged_df['stars_25'])]

# Rename columns to reflect the data properly
star_changes = star_changes[['name', 'address_25', 'location', 'url', 'stars_24', 'stars_25']]

In [59]:
star_changes

,name,address_25,location,url,stars_24,stars_25
24,Georges Blanc,Place du Marché,"Vonnas, 01540",https://www.georgesblanc.com/fr/,3.0,2.0
49,Le Puits Saint Jacques,57 avenue Victor-Capoul,"Pujaudran, 32600",http://www.lepuitssaintjacques.fr,2.0,1.0
73,Christopher Coutanceau,Plage de la Concurrence,"La Rochelle, 17000",http://www.christophercoutanceau.com/fr/,2.0,3.0
110,Maison Nouvelle,11 rue Rode,"Bordeaux, 33000",http://www.maison-nouvelle.fr,1.0,2.0
154,Ekaitza,15 quai Maurice-Ravel,"Ciboure, 64500",https://www.restaurant-ekaitza.fr/,1.0,2.0
171,Rozó,34 rue Raymond-Derain,"Marcq-en-Barœul, 59700",http://www.restaurant-rozo.fr,1.0,2.0
329,L'Auberge de Saint-Rémy - Fanny Rey & Jonathan...,12 boulevard Mirabeau,"Saint-Rémy-de-Provence, 13210",https://www.aubergesaintremy.com/fr/,1.0,2.0
357,L'Observatoire du Gabriel,10 place de la Bourse,"Bordeaux, 33000",https://le-gabriel-bordeaux.fr,1.0,2.0
438,Sushi Yoshinaga,27 rue du 4-Septembre,"Paris, 75002",http://www.sushiyoshinaga.com,1.0,2.0
466,Blanc,52 rue de Longchamp,"Paris, 75116",https://blanc-paris.com/,1.0,2.0


In [60]:
# We have one demotion from 2 star to 1 star to flag as this won't be picked up with analysis of 2 and 3 stars
star_changes.loc[49]

name                     Le Puits Saint Jacques
address_25              57 avenue Victor-Capoul
location                       Pujaudran, 32600
url           http://www.lepuitssaintjacques.fr
stars_24                                    2.0
stars_25                                    1.0
Name: 49, dtype: object

We now need to find new entries at two star and three star

In [61]:
# Convert subset DataFrames to sets of tuples to identify unique restaurants
unique_24 = set(tuple(x) for x in subset_24[['name', 'location', 'stars']].values)
unique_25 = set(tuple(x) for x in subset_25[['name', 'location', 'stars']].values)

# Identify new restaurants in 2024 that were not in 2023
new_restaurants = unique_25 - unique_24

# Convert the set of new restaurants back to a DataFrame
df = pd.DataFrame(list(new_restaurants), columns=['name', 'location', 'stars'])

# Filter for restaurants that have two or three stars
new_high_stars = df[(df['stars'] == 2) | (df['stars'] == 3)]

new_high_stars = new_high_stars.sort_values(by=['stars'], ascending=[False])

In [62]:
new_high_stars

,name,location,stars
5,Christopher Coutanceau,"La Rochelle, 17000",3.0
21,Le Coquillage,"Saint-Méloir-des-Ondes, 35350",3.0
11,La Table des Amis,"Bonnieux, 84480",2.0
20,Cédric Burtin,"Saint-Rémy, 71100",2.0
24,Sushi Yoshinaga,"Paris, 75002",2.0
33,Blanc,"Paris, 75116",2.0
34,Georges Blanc,"Vonnas, 01540",2.0
43,L'Observatoire du Gabriel,"Bordeaux, 33000",2.0
56,L'Auberge de Saint-Rémy - Fanny Rey & Jonathan...,"Saint-Rémy-de-Provence, 13210",2.0
63,Ekaitza,"Ciboure, 64500",2.0


It was [announced in the press](https://guide.michelin.com/gb/en/article/news-and-views/all-the-winners-in-the-michelin-guide-france-2025) that there are two new 3* and nine new 2*.

In [63]:
from fuzzywuzzy import process

# Define a threshold for matching
threshold = 70

# Create a function to perform fuzzy matching
def get_matches(new_entry, old_data):
    # Use fuzzywuzzy's process.extract to get matches with a score
    matches = process.extract(new_entry['name'], old_data['name'], scorer=fuzz.token_set_ratio, limit=None)

    # Filter matches based on a threshold score
    high_score_matches = [match for match in matches if match[1] >= threshold]
    return high_score_matches

In [64]:
# Apply fuzzy matching to the new high-star restaurants
new_high_stars['matches_in_24'] = new_high_stars.apply(lambda x: get_matches(x, france_24), axis=1)

In [65]:
# Display results
new_high_stars[['name', 'location', 'stars', 'matches_in_24']]

,name,location,stars,matches_in_24
5,Christopher Coutanceau,"La Rochelle, 17000",3.0,"[(Christopher Coutanceau, 100, 88)]"
21,Le Coquillage,"Saint-Méloir-des-Ondes, 35350",3.0,"[(Le Coquillage, 100, 96), (Le Cottage, 70, 920)]"
11,La Table des Amis,"Bonnieux, 84480",2.0,"[(La Table des Amis by Christophe Bacquié, 100..."
20,Cédric Burtin,"Saint-Rémy, 71100",2.0,[]
24,Sushi Yoshinaga,"Paris, 75002",2.0,"[(Sushi Yoshinaga, 100, 554), (Sushi B, 83, 386)]"
33,Blanc,"Paris, 75116",2.0,"[(Plénitude - Cheval Blanc Paris, 100, 1), (La..."
34,Georges Blanc,"Vonnas, 01540",2.0,"[(Georges Blanc, 100, 24), (Blanc, 100, 593), ..."
43,L'Observatoire du Gabriel,"Bordeaux, 33000",2.0,"[(L'Observatoire du Gabriel, 100, 451)]"
56,L'Auberge de Saint-Rémy - Fanny Rey & Jonathan...,"Saint-Rémy-de-Provence, 13210",2.0,[(L'Auberge de Saint-Rémy - Fanny Rey & Jonath...
63,Ekaitza,"Ciboure, 64500",2.0,"[(Ekaitza, 100, 195)]"


"Blanc", `index 33` is a false fuzzy-match so we skip

In [66]:
# Create a function to perform fuzzy matching and return the best match above the threshold
def get_best_match(new_entry, old_data):
    # Get all top matches
    matches = process.extract(new_entry['name'], old_data['name'], scorer=fuzz.token_set_ratio, limit=10)

    for name, score, index in matches:
        # If the score is perfect and it's an exact name match, use it immediately
        if score == 100 and name.strip().lower() == new_entry['name'].strip().lower():
            return (name, old_data.iloc[index]['location']), score, old_data.iloc[index]['stars']

    # Otherwise, return the top match if score passes threshold
    best_match, score, index = matches[0]
    if score >= threshold:
        return (old_data.iloc[index]['name'], old_data.iloc[index]['location']), score, old_data.iloc[index]['stars']
    else:
        return None, None, None

In [67]:
# Apply fuzzy matching to the new high-star restaurants
new_high_stars[['best_match_24', 'match_score_24', 'stars_24']] = new_high_stars.apply(
    lambda x: get_best_match(x, france_24),
    axis=1, result_type='expand'
)

# Display results
new_high_stars[['name', 'location', 'stars', 'stars_24', 'best_match_24']]

,name,location,stars,stars_24,best_match_24
5,Christopher Coutanceau,"La Rochelle, 17000",3.0,2.0,"(Christopher Coutanceau, La Rochelle, 17000)"
21,Le Coquillage,"Saint-Méloir-des-Ondes, 35350",3.0,2.0,"(Le Coquillage, Saint-Méloir-des-Ondes, 35350)"
11,La Table des Amis,"Bonnieux, 84480",2.0,2.0,"(La Table des Amis by Christophe Bacquié, Bonn..."
20,Cédric Burtin,"Saint-Rémy, 71100",2.0,NaN,None
24,Sushi Yoshinaga,"Paris, 75002",2.0,1.0,"(Sushi Yoshinaga, Paris, 75002)"
33,Blanc,"Paris, 75116",2.0,1.0,"(Blanc, Paris, 75116)"
34,Georges Blanc,"Vonnas, 01540",2.0,3.0,"(Georges Blanc, Vonnas, 01540)"
43,L'Observatoire du Gabriel,"Bordeaux, 33000",2.0,1.0,"(L'Observatoire du Gabriel, Bordeaux, 33000)"
56,L'Auberge de Saint-Rémy - Fanny Rey & Jonathan...,"Saint-Rémy-de-Provence, 13210",2.0,1.0,(L'Auberge de Saint-Rémy - Fanny Rey & Jonatha...
63,Ekaitza,"Ciboure, 64500",2.0,1.0,"(Ekaitza, Ciboure, 64500)"


We define a function to return the status of the restaurant after inspection of the above df

In [68]:
# Define a function to determine the status of the restaurant
def determine_status(row):
    # Extract the postal code from the location and best_match_23
    postal_code = row['location'].split(', ')[-1]
    match_postal_code = row['best_match_24'][1].split(', ')[-1] if row['best_match_24'] else None

    # Check if the postal codes match
    if postal_code != match_postal_code:
        return 'new award'
    else:
        # Check for promotion or demotion
        if row['stars_24'] and row['stars']:
            if row['stars_24'] < row['stars']:
                return 'promoted'
            elif row['stars_24'] > row['stars']:
                return 'demoted'
        # If stars are the same or no match, return None (for removal)
        return None

In [69]:
# Cast 'stars_24' to object
new_high_stars['stars_24'] = new_high_stars['stars_24'].astype('object')
# Apply the function to each row in the DataFrame
new_high_stars['status'] = new_high_stars.apply(determine_status, axis=1)

# update the 'stars_23' based on the status
new_high_stars.loc[new_high_stars['status'] == 'new award', 'stars_24'] = 'New Entry'

# Remove the rows where the status is None (meaning no change in stars)
new_high_stars = new_high_stars.dropna(subset=['status'])

# Display the updated DataFrame
major_star_changes = new_high_stars[['name', 'location', 'stars', 'stars_24', 'status']]

In [70]:
major_star_changes

,name,location,stars,stars_24,status
5,Christopher Coutanceau,"La Rochelle, 17000",3.0,2.0,promoted
21,Le Coquillage,"Saint-Méloir-des-Ondes, 35350",3.0,2.0,promoted
20,Cédric Burtin,"Saint-Rémy, 71100",2.0,New Entry,new award
24,Sushi Yoshinaga,"Paris, 75002",2.0,1.0,promoted
33,Blanc,"Paris, 75116",2.0,1.0,promoted
34,Georges Blanc,"Vonnas, 01540",2.0,3.0,demoted
43,L'Observatoire du Gabriel,"Bordeaux, 33000",2.0,1.0,promoted
56,L'Auberge de Saint-Rémy - Fanny Rey & Jonathan...,"Saint-Rémy-de-Provence, 13210",2.0,1.0,promoted
63,Ekaitza,"Ciboure, 64500",2.0,1.0,promoted
71,Baumanière 1850,"Courchevel, 73120",2.0,1.0,promoted


In [71]:
# We manually write the demotion of Le Puits Saint Jacques to star_changes
puits_StJ = star_changes.loc[49]
puits_StJ

name                     Le Puits Saint Jacques
address_25              57 avenue Victor-Capoul
location                       Pujaudran, 32600
url           http://www.lepuitssaintjacques.fr
stars_24                                    2.0
stars_25                                    1.0
Name: 49, dtype: object

In [72]:
data_to_append = {
    'name': puits_StJ['name'],
    'location': puits_StJ['location'],
    'stars': puits_StJ['stars_25'],
    'stars_24': puits_StJ['stars_24'],
    'status': 'demoted'
}

row_to_append = pd.DataFrame([data_to_append])
major_star_changes = pd.concat([major_star_changes, row_to_append], ignore_index=True)

In [73]:
major_star_changes

,name,location,stars,stars_24,status
0,Christopher Coutanceau,"La Rochelle, 17000",3.0,2.0,promoted
1,Le Coquillage,"Saint-Méloir-des-Ondes, 35350",3.0,2.0,promoted
2,Cédric Burtin,"Saint-Rémy, 71100",2.0,New Entry,new award
3,Sushi Yoshinaga,"Paris, 75002",2.0,1.0,promoted
4,Blanc,"Paris, 75116",2.0,1.0,promoted
5,Georges Blanc,"Vonnas, 01540",2.0,3.0,demoted
6,L'Observatoire du Gabriel,"Bordeaux, 33000",2.0,1.0,promoted
7,L'Auberge de Saint-Rémy - Fanny Rey & Jonathan...,"Saint-Rémy-de-Provence, 13210",2.0,1.0,promoted
8,Ekaitza,"Ciboure, 64500",2.0,1.0,promoted
9,Baumanière 1850,"Courchevel, 73120",2.0,1.0,promoted


----
&nbsp;
### Merging the promotions/demotions with `france_25`

In [74]:
france_25.head()

,name,address,location,arrondissement,department_num,department,capital,region,price,cuisine,url,award,stars,greenstar,longitude,latitude
0,L'Ambroisie,9 place des Vosges,"Paris, 75004",4th (Hôtel-de-Ville),75,Paris,Paris,Île-de-France,€€€€,Classic Cuisine,https://www.ambroisie-paris.com/,3 Stars,3.0,0,2.364521,48.855494
1,Flocons de Sel,"1775 route du Leutaz, Le Leutaz","Megève, 74120",Bonneville,74,Haute-Savoie,Annecy,Auvergne-Rhône-Alpes,€€€€,Modern Cuisine,https://www.floconsdesel.com,3 Stars,3.0,0,6.596963,45.830240
2,Épicure,"Le Bristol, 112 rue du Faubourg-Saint-Honoré","Paris, 75008",8th (Élysée),75,Paris,Paris,Île-de-France,€€€€,Modern Cuisine,https://www.oetkercollection.com/fr/hotels/le-...,3 Stars,3.0,0,2.314598,48.871722
3,Arpège,84 rue de Varenne,"Paris, 75007",7th (Palais-Bourbon),75,Paris,Paris,Île-de-France,€€€€,Creative,https://www.alain-passard.com/,3 Stars,3.0,1,2.317013,48.855754
4,Le Pré Catelan,Route de Suresnes - bois de Boulogne,"Paris, 75016",16th (Passy),75,Paris,Paris,Île-de-France,€€€€,"Creative, Contemporary",https://www.leprecatelan.com/,3 Stars,3.0,0,2.250718,48.863937


In [75]:
# Merge major_star_changes with france_24 on 'name' and 'location'
merged_changes = pd.merge(major_star_changes, france_25, on=['name', 'location'], how='left')

merged_changes.rename(columns={'stars_y': 'stars'}, inplace=True)
column_order = ['name', 'address', 'location', 'arrondissement', 'department_num', 'department', 'capital', 'region',
       'price', 'cuisine', 'url', 'award', 'stars', 'stars_24', 'status', 'longitude', 'latitude']
merged_changes = merged_changes[column_order]

In [76]:
merged_changes

,name,address,location,arrondissement,department_num,department,capital,region,price,cuisine,url,award,stars,stars_24,status,longitude,latitude
0,Christopher Coutanceau,Plage de la Concurrence,"La Rochelle, 17000",La Rochelle,17,Charente-Maritime,La Rochelle,Nouvelle-Aquitaine,€€€€,"Seafood, Modern Cuisine",http://www.christophercoutanceau.com/fr/,3 Stars,3.0,2.0,promoted,-1.159918,46.155363
1,Le Coquillage,Lieu-dit Le Buot,"Saint-Méloir-des-Ondes, 35350",Saint-Malo,35,Ille-et-Vilaine,Rennes,Bretagne,€€€€,"Creative, Seafood",https://www.maisons-de-bricourt.com/fr/page/le...,3 Stars,3.0,2.0,promoted,-1.871123,48.643065
2,Cédric Burtin,Chemin de Martorez,"Saint-Rémy, 71100",Chalon-sur-Saône,71,Saône-et-Loire,Mâcon,Bourgogne-Franche-Comté,€€€€,"Creative, Modern Cuisine",https://cedricburtin.com/fr/,2 Stars,2.0,New Entry,new award,4.820027,46.756771
3,Sushi Yoshinaga,27 rue du 4-Septembre,"Paris, 75002",2nd (Bourse),75,Paris,Paris,Île-de-France,€€€€,Japanese,http://www.sushiyoshinaga.com,2 Stars,2.0,1.0,promoted,2.334956,48.869838
4,Blanc,52 rue de Longchamp,"Paris, 75116",16th (Passy),75,Paris,Paris,Île-de-France,€€€€,Creative,https://blanc-paris.com/,2 Stars,2.0,1.0,promoted,2.287481,48.865117
5,Georges Blanc,Place du Marché,"Vonnas, 01540",Bourg-en-Bresse,01,Ain,Bourg-en-Bresse,Auvergne-Rhône-Alpes,€€€€,Classic Cuisine,https://www.georgesblanc.com/fr/,2 Stars,2.0,3.0,demoted,4.989499,46.220108
6,L'Observatoire du Gabriel,10 place de la Bourse,"Bordeaux, 33000",Bordeaux,33,Gironde,Bordeaux,Nouvelle-Aquitaine,€€€€,"Modern Cuisine, Creative",https://le-gabriel-bordeaux.fr,2 Stars,2.0,1.0,promoted,-0.570715,44.841350
7,L'Auberge de Saint-Rémy - Fanny Rey & Jonathan...,12 boulevard Mirabeau,"Saint-Rémy-de-Provence, 13210",Arles,13,Bouches-du-Rhône,Marseille,Provence-Alpes-Côte d'Azur,€€€€,Modern Cuisine,https://www.aubergesaintremy.com/fr/,2 Stars,2.0,1.0,promoted,4.833321,43.788561
8,Ekaitza,15 quai Maurice-Ravel,"Ciboure, 64500",Bayonne,64,Pyrénées-Atlantiques,Pau,Nouvelle-Aquitaine,€€€,"Creative, Modern Cuisine",https://www.restaurant-ekaitza.fr/,2 Stars,2.0,1.0,promoted,-1.667464,43.385259
9,Baumanière 1850,"Le Strato, Route de Bellecôte, Courchevel 1850","Courchevel, 73120",Albertville,73,Savoie,Chambéry,Auvergne-Rhône-Alpes,€€€€,Creative,https://www.hotelstrato.com/,2 Stars,2.0,1.0,promoted,6.638078,45.410293


----
&nbsp;
## Changes in the michelin guide

In [77]:
stars_24 = france_24[france_24['stars'] >= 1]
stars_25 = france_25[france_25['stars'] >= 1]

In [78]:
# Count the occurrences of each star in france_23 and france_24
stars_count_24 = stars_24.groupby('stars').size()
stars_count_25 = stars_25.groupby('stars').size()

# Ensure all possible star values are represented, even if there are no restaurants with that count
star_ratings = [1, 2, 3]
stars_count_24 = stars_count_24.reindex(star_ratings)
stars_count_25 = stars_count_25.reindex(star_ratings)

# Create a DataFrame to display the counts side by side
star_comparison_table = pd.DataFrame({
    'Star Rating': star_ratings,
    '2024 Guide': stars_count_24.values,
    '2025 Guide': stars_count_25.values
})

# Display the table
star_comparison_table

,Star Rating,2024 Guide,2025 Guide
0,1,530,538
1,2,73,78
2,3,29,30


----
&nbsp;
## New restaurants by region

In [79]:
# Function import
from Functions.functions_visualisation import top_restaurants

In [80]:
top_restaurants(merged_changes, granularity='region', star_rating=3, top_n=5)

Only 2 unique regions found.

Top 2 regions with most ⭐⭐⭐ restaurants:


Region: Bretagne
1 ⭐⭐⭐ Restaurant





Region: Nouvelle-Aquitaine
1 ⭐⭐⭐ Restaurant




In [81]:
top_restaurants(merged_changes, granularity='region', star_rating=2, top_n=5)

Top 5 regions with most ⭐⭐ restaurants:


Region: Nouvelle-Aquitaine
3 ⭐⭐ Restaurants





Region: Auvergne-Rhône-Alpes
2 ⭐⭐ Restaurants





Region: Île-de-France
2 ⭐⭐ Restaurants





Region: Bourgogne-Franche-Comté
1 ⭐⭐ Restaurant





Region: Hauts-de-France
1 ⭐⭐ Restaurant




----
&nbsp;
## Postscript - Useful code
Searching for particular restaurants - Verify the differences

In [82]:
# Function to find potential matches based on the restaurant name
def find_potential_matches(search_name, df, threshold=70):
    # Use fuzzy matching to find potential matches and their scores
    potential_matches = process.extract(search_name, df['name'], scorer=fuzz.token_set_ratio)
    
    # Filter out matches that meet or exceed the threshold score
    good_matches = [(match[0], match[1]) for match in potential_matches if match[1] >= threshold]
    return good_matches

In [85]:
# The name of the restaurant you're interested in
search_name = "Le Coquillage"

In [86]:
# Use the function to find matches in france_23
matches_in_24 = find_potential_matches(search_name, france_24, threshold)

# Use the function to find matches in france_24
matches_in_25 = find_potential_matches(search_name, france_25, threshold)

# Display the results
print("Potential matches in 2024 data:")
for match in matches_in_24:
    print(f"Name: {match[0]}, Score: {match[1]}")

print("\nPotential matches in 2025 data:")
for match in matches_in_25:
    print(f"Name: {match[0]}, Score: {match[1]}")

Potential matches in 2024 data:
Name: Le Coquillage, Score: 100
Name: Le Cottage, Score: 70

Potential matches in 2025 data:
Name: Le Coquillage, Score: 100
Name: Le Collet, Score: 73
Name: La Courtille, Score: 72
Name: Le Cottage, Score: 70


----